In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!wandb login 9676e3cc95066e4865586082971f2653245f09b4

In [ ]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from scipy.special import factorial

from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib import path as mpath
import matplotlib.gridspec as gridspec

import pickle
import tabulate
import wandb
from collections import namedtuple

import meta_learning_data_analysis as analysis
import meta_learning_analysis_plots as plots

In [ ]:
np.set_printoptions(linewidth=160)
# Avoid escaping any latex characters
tabulate.LATEX_ESCAPE_RULES = {}

MIN_FONT_SIZE = 14
LABELS_FONT_SIZE = 20
TITLE_FONT_SIZE = 24

matplotlib.rcParams.update({'font.size': MIN_FONT_SIZE})
FONT_DICT = dict(fontsize=LABELS_FONT_SIZE) 
TITLE_FONT_DICT = dict(fontsize=TITLE_FONT_SIZE)

PER_PLOT_WIDTH = 6

QUERY_ORDINAL_POSITION_COMORLAP= 'cool'
NUM_TIMES_TRAINED_COLORMAP = 'copper'

In [ ]:
cache = analysis.refresh_cache()
print(cache.keys())

# query_mod_replications = cache['query_mod_replications']
control_analyses = cache['control_analyses']
six_replications_updated_analyses = cache['six_replications_updated_analyses']
control_updated_analyses = cache['updated_control_analyses']
query_mod_updated_analyses = cache['query_mod_updated_analyses']

# MAML
maml_analyses = cache['maml_analyses']
maml_meta_test_analyses = cache['maml_meta_test_analyses']
balanced_batches_analyses = cache['balanced_batches_analyses']

-----

# Basic plots for regular data, and MAML-related conditions



In [ ]:
NROWS = 1
NCOLS = 2
COL_WIDTH = 6
ROW_HEIGHT = 5 
WIDTH_SPACING = 2
HEIGHT_SPACING = 0

title_font_dict = TITLE_FONT_DICT.copy()
title_font_dict['pad'] = 10

multi_plot_font_dict = FONT_DICT.copy()
multi_plot_font_dict['fontsize'] = 18
    
figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
plt.subplots_adjust(hspace=0.4, wspace=0.4)

replication_n = np.array((10, 10, 10, 30))
LOG_PLOT_YLIM = (7.75, 13.25)

comparison_level = 0

replication_n = np.array((10, 10, 10, 30))
sem_n = replication_n * 2

axes = (plt.subplot(NROWS, NCOLS, 1), None) # plt.subplot(NROWS, NCOLS, NCOLS + 1))
plots.plot_processed_results_all_dimensions(maml_meta_test_analyses, data_index=1, 
                                            title='', num_times_trained_title='',
                                            ylim=LOG_PLOT_YLIM, log_x=True, log_y=True, sem_n=sem_n, shade_error=True,
                                            font_dict=multi_plot_font_dict, times_trained_colormap=QUERY_ORDINAL_POSITION_COMORLAP, 
                                            tasks_trained_colormap=NUM_TIMES_TRAINED_COLORMAP,
                                            dimension_names=('',), dimension_indices=(3,),
                                            num_tasks_trained_highlight_first_time='highlight', add_colorbars=False,
                                            title_font_dict=TITLE_FONT_DICT, external_axes=axes,
                                            plot_regression='print')

axes = (plt.subplot(NROWS, NCOLS, 2), None) # plt.subplot(NROWS, NCOLS, NCOLS + 2))

plots.combined_comparison_plots(balanced_batches_analyses, {1: data}, 
                            '',
                            comparison_level, comparison_func=np.subtract,
                            font_dict=multi_plot_font_dict, comparison_first=True, 
                            data_index=0, 
                            log_x=False, log_y=False, shade_error=False,
                            title_font_dict=TITLE_FONT_DICT,
                            times_trained_colormap=QUERY_ORDINAL_POSITION_COMORLAP, 
                            tasks_trained_colormap=NUM_TIMES_TRAINED_COLORMAP,
                            plot_y_label='$\Delta$ trials to criterion\n(Positive = MAML better)',
                            y_custom_tick_labels=[f'{int(x)}{x != 0 and "k" or ""}' for x in np.arange(-7, 2) * 25],
                            external_axes=axes, 
                            replication_levels=(1,), custom_titles=('',))

TEXT_POSITIONS = [(0.045, 0.83), (0.475, 0.83)]

subfigure_text_font_dict = FONT_DICT.copy()
subfigure_text_font_dict['fontsize'] += 4
subfigure_text_font_dict['color'] = '#808080'
subfigure_text_font_dict['weight'] = 'bold'

for i in range(len(TEXT_POSITIONS)):
    pos = TEXT_POSITIONS[i]
    plt.text(pos[0], pos[1], f'({chr(97 + i)})', 
             subfigure_text_font_dict, transform=figure.transFigure)

plots.save('nips/response_maml_panel.pdf')

In [ ]:
[f'{int(x)}{x != 0 and "k" or ""}' for x in np.arange(-6, 2) * 25]

## Examples to criterion

In [ ]:
NROWS = 2
NCOLS = 4
COL_WIDTH = 6
ROW_HEIGHT = 5 
WIDTH_SPACING = 2
HEIGHT_SPACING = 0

title_font_dict = TITLE_FONT_DICT.copy()
title_font_dict['pad'] = 10

multi_plot_font_dict = FONT_DICT.copy()
multi_plot_font_dict['fontsize'] = 18
    
figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
plt.subplots_adjust(hspace=0.4, wspace=0.35)

replication_n = np.array((10, 10, 10, 30))
LOG_PLOT_YLIM = (7.75, 13.25)

for index, (data, sem_n, title) in enumerate(zip((six_replications_updated_analyses, balanced_batches_analyses,
                                                 maml_analyses, maml_meta_test_analyses),
                                                 (replication_n * 6, replication_n * 3,
                                                  replication_n * 3, replication_n * 2),
                                                 ('Baseline', 'Balanced batches',
                                                  'MAML', 'MAML (meta-test)'))):
    axes = (plt.subplot(NROWS, NCOLS, index + 1), plt.subplot(NROWS, NCOLS, NCOLS + index + 1))
    plots.plot_processed_results_all_dimensions(data, data_index=1, 
                                                title='', num_times_trained_title=title,
                                                ylim=LOG_PLOT_YLIM, log_x=True, log_y=True, sem_n=sem_n, shade_error=True,
                                                font_dict=multi_plot_font_dict, times_trained_colormap=QUERY_ORDINAL_POSITION_COMORLAP, 
                                                tasks_trained_colormap=NUM_TIMES_TRAINED_COLORMAP,
                                                dimension_names=('',), dimension_indices=(3,),
                                                num_tasks_trained_highlight_first_time='highlight', add_colorbars=False,
                                                title_font_dict=TITLE_FONT_DICT, external_axes=axes,
                                                plot_regression='print')



plt.show()

-----

## Playing around with difference plots
Like what we did for the heterogeneous dimensions condition. Subtracting the baseline from each of the three other conditions above.

### First case -- subtracting in the raw example space

In [ ]:
NROWS = 2
NCOLS = 3
COL_WIDTH = 6
ROW_HEIGHT = 5 
WIDTH_SPACING = 2
HEIGHT_SPACING = 0

title_font_dict = TITLE_FONT_DICT.copy()
title_font_dict['pad'] = 10

multi_plot_font_dict = FONT_DICT.copy()
multi_plot_font_dict['fontsize'] = 18
    
figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
plt.subplots_adjust(hspace=0.4, wspace=0.7)

replication_n = np.array((10, 10, 10, 30))
LOG_PLOT_YLIM = (7.75, 13.25)

comparison_level = 0

for index, (data, sem_n, title) in enumerate(zip((six_replications_updated_analyses,
                                                 maml_analyses, maml_meta_test_analyses),
                                                 (replication_n * 6,
                                                  replication_n * 3, replication_n * 2),
                                                 ('Baseline',
                                                  'MAML', 'MAML (meta-test)'))):
    axes = (plt.subplot(NROWS, NCOLS, index + 1), plt.subplot(NROWS, NCOLS, NCOLS + index + 1))

    plots.combined_comparison_plots(balanced_batches_analyses, {1: data}, 
                                '',
                                comparison_level, comparison_func=np.subtract,
                                font_dict=multi_plot_font_dict, comparison_first=True, 
                                data_index=0, 
                                log_x=False, log_y=False, shade_error=False,
                                title_font_dict=TITLE_FONT_DICT,
                                times_trained_colormap=QUERY_ORDINAL_POSITION_COMORLAP, 
                                tasks_trained_colormap=NUM_TIMES_TRAINED_COLORMAP,
                                plot_y_label='$\Delta$ trials to criterion\n(Positive = MAML better)',
                                # y_custom_tick_labels=[None, '-50k', '0'] + [f'{(i+1) * 50}k' for i in range(4)],
                                external_axes=axes, 
                                replication_levels=(1,), custom_titles=(title,))

### Second case -- dividing in the raw trials space

In [ ]:
NROWS = 2
NCOLS = 3
COL_WIDTH = 6
ROW_HEIGHT = 5 
WIDTH_SPACING = 2
HEIGHT_SPACING = 0

title_font_dict = TITLE_FONT_DICT.copy()
title_font_dict['pad'] = 10

multi_plot_font_dict = FONT_DICT.copy()
multi_plot_font_dict['fontsize'] = 18
    
figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
plt.subplots_adjust(hspace=0.4, wspace=0.6)

replication_n = np.array((10, 10, 10, 30))
LOG_PLOT_YLIM = (7.75, 13.25)

comparison_level = 0

for index, (data, sem_n, title) in enumerate(zip((six_replications_updated_analyses,
                                                 maml_analyses, maml_meta_test_analyses),
                                                 (replication_n * 6,
                                                  replication_n * 3, replication_n * 2),
                                                 ('Baseline',
                                                  'MAML', 'MAML (meta-test)'))):
    axes = (plt.subplot(NROWS, NCOLS, index + 1), plt.subplot(NROWS, NCOLS, NCOLS + index + 1))

    plots.combined_comparison_plots(balanced_batches_analyses, {1: data},
                                '',
                                comparison_level, comparison_func=np.divide,
                                font_dict=multi_plot_font_dict, comparison_first=True, 
                                data_index=0, 
                                log_x=False, log_y=False, shade_error=False,
                                title_font_dict=TITLE_FONT_DICT,
                                times_trained_colormap=QUERY_ORDINAL_POSITION_COMORLAP, 
                                tasks_trained_colormap=NUM_TIMES_TRAINED_COLORMAP,
                                plot_y_label='Ratio trials to criterion',
                                # y_custom_tick_labels=[None, '-50k', '0'] + [f'{(i+1) * 50}k' for i in range(4)],
                                external_axes=axes, 
                                replication_levels=(1,), custom_titles=(title,))

### Second case -- division in the raw trials space -- flipped

In [ ]:
NROWS = 2
NCOLS = 3
COL_WIDTH = 6
ROW_HEIGHT = 5 
WIDTH_SPACING = 2
HEIGHT_SPACING = 0

title_font_dict = TITLE_FONT_DICT.copy()
title_font_dict['pad'] = 10

multi_plot_font_dict = FONT_DICT.copy()
multi_plot_font_dict['fontsize'] = 18
    
figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
plt.subplots_adjust(hspace=0.4, wspace=0.6)

replication_n = np.array((10, 10, 10, 30))
LOG_PLOT_YLIM = (7.75, 13.25)

comparison_level = 0

for index, (data, sem_n, title) in enumerate(zip((six_replications_updated_analyses,
                                                 maml_analyses, maml_meta_test_analyses),
                                                 (replication_n * 6,
                                                  replication_n * 3, replication_n * 2),
                                                 ('Baseline',
                                                  'MAML', 'MAML (meta-test)'))):
    axes = (plt.subplot(NROWS, NCOLS, index + 1), plt.subplot(NROWS, NCOLS, NCOLS + index + 1))

    plots.combined_comparison_plots(balanced_batches_analyses, {1: data}, 
                                '',
                                comparison_level, comparison_func=np.divide,
                                font_dict=multi_plot_font_dict, comparison_first=False, 
                                data_index=0, 
                                log_x=False, log_y=False, shade_error=False,
                                title_font_dict=TITLE_FONT_DICT,
                                times_trained_colormap=QUERY_ORDINAL_POSITION_COMORLAP, 
                                tasks_trained_colormap=NUM_TIMES_TRAINED_COLORMAP,
                                plot_y_label='Ratio trials to criterion',
                                # y_custom_tick_labels=[None, '-50k', '0'] + [f'{(i+1) * 50}k' for i in range(4)],
                                external_axes=axes, 
                                replication_levels=(1,), custom_titles=(title,))

### Third case - substracting in log space

In [ ]:
NROWS = 2
NCOLS = 3
COL_WIDTH = 6
ROW_HEIGHT = 5 
WIDTH_SPACING = 2
HEIGHT_SPACING = 0

title_font_dict = TITLE_FONT_DICT.copy()
title_font_dict['pad'] = 10

multi_plot_font_dict = FONT_DICT.copy()
multi_plot_font_dict['fontsize'] = 18
    
figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
plt.subplots_adjust(hspace=0.4, wspace=0.6)

replication_n = np.array((10, 10, 10, 30))
LOG_PLOT_YLIM = (7.75, 13.25)

comparison_level = 0

for index, (data, sem_n, title) in enumerate(zip((six_replications_updated_analyses,
                                                 maml_analyses, maml_meta_test_analyses),
                                                 (replication_n * 6,
                                                  replication_n * 3, replication_n * 2),
                                                 ('Baseline',
                                                  'MAML', 'MAML (meta-test)'))):
    axes = (plt.subplot(NROWS, NCOLS, index + 1), plt.subplot(NROWS, NCOLS, NCOLS + index + 1))

    plots.combined_comparison_plots(balanced_batches_analyses, {1: data}, 
                                '',
                                comparison_level, comparison_func=np.subtract,
                                font_dict=multi_plot_font_dict, comparison_first=True, 
                                data_index=1, 
                                log_x=False, log_y=False, shade_error=False,
                                title_font_dict=TITLE_FONT_DICT,
                                times_trained_colormap=QUERY_ORDINAL_POSITION_COMORLAP, 
                                tasks_trained_colormap=NUM_TIMES_TRAINED_COLORMAP,
                                plot_y_label='$\Delta$ trials to criterion',
                                # y_custom_tick_labels=[None, '-50k', '0'] + [f'{(i+1) * 50}k' for i in range(4)],
                                external_axes=axes, 
                                replication_levels=(1,), custom_titles=(title,))

### Fourth case - dividing in log space

In [ ]:
NROWS = 2
NCOLS = 3
COL_WIDTH = 6
ROW_HEIGHT = 5 
WIDTH_SPACING = 2
HEIGHT_SPACING = 0

title_font_dict = TITLE_FONT_DICT.copy()
title_font_dict['pad'] = 10

multi_plot_font_dict = FONT_DICT.copy()
multi_plot_font_dict['fontsize'] = 18
    
figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
plt.subplots_adjust(hspace=0.4, wspace=0.6)

replication_n = np.array((10, 10, 10, 30))
LOG_PLOT_YLIM = (7.75, 13.25)

comparison_level = 0

for index, (data, sem_n, title) in enumerate(zip((six_replications_updated_analyses,
                                                 maml_analyses, maml_meta_test_analyses),
                                                 (replication_n * 6,
                                                  replication_n * 3, replication_n * 2),
                                                 ('Baseline',
                                                  'MAML', 'MAML (meta-test)'))):
    axes = (plt.subplot(NROWS, NCOLS, index + 1), plt.subplot(NROWS, NCOLS, NCOLS + index + 1))

    plots.combined_comparison_plots(balanced_batches_analyses, {1: data}, 
                                '',
                                comparison_level, comparison_func=np.divide,
                                font_dict=multi_plot_font_dict, comparison_first=True, 
                                data_index=1, 
                                log_x=False, log_y=False, shade_error=False,
                                title_font_dict=TITLE_FONT_DICT,
                                times_trained_colormap=QUERY_ORDINAL_POSITION_COMORLAP, 
                                tasks_trained_colormap=NUM_TIMES_TRAINED_COLORMAP,
                                plot_y_label='Ratio trials to criterion',
                                # y_custom_tick_labels=[None, '-50k', '0'] + [f'{(i+1) * 50}k' for i in range(4)],
                                external_axes=axes, 
                                replication_levels=(1,), custom_titles=(title,))

-------

## Accuracy after 22.5k examples

In [ ]:
NROWS = 2
NCOLS = 4
COL_WIDTH = 6
ROW_HEIGHT = 5 
WIDTH_SPACING = 2
HEIGHT_SPACING = 0

title_font_dict = TITLE_FONT_DICT.copy()
title_font_dict['pad'] = 10

multi_plot_font_dict = FONT_DICT.copy()
multi_plot_font_dict['fontsize'] = 18
    
figure = plt.figure(figsize=(NCOLS * COL_WIDTH + WIDTH_SPACING, NROWS * ROW_HEIGHT + HEIGHT_SPACING))
plt.subplots_adjust(hspace=0.4, wspace=0.35)

replication_n = np.array((10, 10, 10, 30))
ACCURACY_PLOT_YLIM = (0.5, 1.0)

for index, (data, sem_n, title) in enumerate(zip((six_replications_updated_analyses, balanced_batches_analyses,
                                                 maml_analyses, maml_meta_test_analyses),
                                                 (replication_n * 6, replication_n * 3,
                                                  replication_n * 3, replication_n * 2),
                                                 ('Baseline', 'Balanced batches',
                                                  'MAML', 'MAML (meta-test)'))):
    axes = (plt.subplot(NROWS, NCOLS, index + 1), plt.subplot(NROWS, NCOLS, NCOLS + index + 1))
    plots.plot_processed_results_all_dimensions(data, data_index=3, 
                                                title='', num_times_trained_title=title,
                                                ylim=ACCURACY_PLOT_YLIM, sem_n=sem_n, shade_error=True, plot_y_label='Accuracy\nafter 22.5k trials',
                                                font_dict=multi_plot_font_dict, times_trained_colormap=QUERY_ORDINAL_POSITION_COMORLAP, 
                                                tasks_trained_colormap=NUM_TIMES_TRAINED_COLORMAP,
                                                dimension_names=('',), dimension_indices=(3,),
                                                num_tasks_trained_highlight_first_time='highlight', add_colorbars=False,
                                                title_font_dict=TITLE_FONT_DICT, external_axes=axes,
                                                plot_regression='print')



plt.show()


----

Trying some sign tests

In [ ]:



diff = six_replications_updated_analyses.combined.log_examples.mean - maml_meta_test_analyses.combined.log_examples.mean

plt.imshow(diff, 'bwr')
plt.colorbar()